In [1]:
@file:Repository("https://repo.kotlin.link")
@file:DependsOn("space.kscience:kmath-tensors-jvm:0.3.0-dev-8")

In [2]:
import space.kscience.kmath.tensors.core.*
import space.kscience.kmath.operations.invoke
import kotlin.math.sqrt 

# Neural network

In [3]:
interface Layer {
    fun forward(input: DoubleTensor): DoubleTensor
    fun backward(input: DoubleTensor, outputError: DoubleTensor): DoubleTensor
}

In [4]:
interface Activation : Layer {
    
    val activation: (DoubleTensor) -> DoubleTensor
    val activationDer: (DoubleTensor) -> DoubleTensor
    
    override fun forward(input: DoubleTensor): DoubleTensor {
        return activation(input)
    }

    override fun backward(input: DoubleTensor, outputError: DoubleTensor): DoubleTensor {
        return DoubleTensorAlgebra { outputError * activationDer(input) }
    }
}

In [5]:
class ReLU : Activation {
    fun relu(x: DoubleTensor): DoubleTensor = DoubleTensorAlgebra {
        x.map { if (it > 0) it else 0.0 }
    }

    fun reluDer(x: DoubleTensor): DoubleTensor = DoubleTensorAlgebra {
        x.map { if (it > 0) 1.0 else 0.0 }
    }

    override val activation = ::relu
    override val activationDer = ::reluDer
}

In [6]:
class Sigmoid : Activation {
    fun sigmoid(x: DoubleTensor): DoubleTensor = DoubleTensorAlgebra {
        1.0 / (1.0 + (-x).exp())
    }

    fun sigmoidDer(x: DoubleTensor): DoubleTensor = DoubleTensorAlgebra {
        sigmoid(x) * (1.0 - sigmoid(x))
    }

    override val activation = ::sigmoid
    override val activationDer = ::sigmoidDer
}

In [7]:
class Dense(
    private val inputUnits: Int,
    private val outputUnits: Int,
    private val learningRate: Double = 0.1
) : Layer {

    private val weights: DoubleTensor = DoubleTensorAlgebra {
        randomNormal(
            intArrayOf(inputUnits, outputUnits)
        ) * kotlin.math.sqrt(2.0 / (inputUnits + outputUnits))
    }

    private val bias: DoubleTensor = DoubleTensorAlgebra { zeros(intArrayOf(outputUnits)) }

    override fun forward(input: DoubleTensor): DoubleTensor {
        return BroadcastDoubleTensorAlgebra { (input dot weights) + bias }
    }

    override fun backward(input: DoubleTensor, outputError: DoubleTensor): DoubleTensor = DoubleTensorAlgebra {
        val gradInput = outputError dot weights.transpose()

        val gradW = input.transpose() dot outputError
        val gradBias = outputError.mean(dim = 0, keepDim = false) * input.shape[0].toDouble()

        weights -= learningRate * gradW
        bias -= learningRate * gradBias

        gradInput
    }

}

In [8]:
class NeuralNetwork(private val layers: List<Layer>) {
    
    private fun softMaxLossGrad(yPred: DoubleTensor, yTrue: DoubleTensor): DoubleTensor = BroadcastDoubleTensorAlgebra {

        val onesForAnswers = yPred.zeroesLike()
        yTrue.toDoubleArray().forEachIndexed { index, labelDouble ->
            val label = labelDouble.toInt()
            onesForAnswers[intArrayOf(index, label)] = 1.0
        }

        val softmaxValue =  yPred.exp() / yPred.exp().sum(dim = 1, keepDim = true)

        (-onesForAnswers + softmaxValue) / (yPred.shape[0].toDouble())
    }


    private fun forward(x: DoubleTensor): List<DoubleTensor> {
        var input = x
        
        val outputs = mutableListOf<DoubleTensor>() 
        
        layers.forEach { layer ->
                val output = layer.forward(input)
                outputs.add(output)
                input = output
            }

        return outputs 
    }

    private fun train(xTrain: DoubleTensor, yTrain: DoubleTensor) {
        val layerInputs = mutableListOf<DoubleTensor>(xTrain)
        
        layerInputs.addAll(forward(xTrain))
        
        var lossGrad = softMaxLossGrad(layerInputs.last(), yTrain)

        layers.zip(layerInputs).reversed().forEach { (layer, input) ->
            lossGrad = layer.backward(input, lossGrad)
        }
    }
    
    fun accuracy(yPred: DoubleTensor, yTrue: DoubleTensor): Double {
        check(yPred.shape contentEquals yTrue.shape)
        val n = yPred.shape[0]
        var correctCnt = 0
        for (i in 0 until n) {
            if (yPred[intArrayOf(i, 0)] == yTrue[intArrayOf(i, 0)]) {
                correctCnt += 1
            }
        }
        return correctCnt.toDouble() / n.toDouble()
    }

    fun fit(xTrain: DoubleTensor, yTrain: DoubleTensor, batchSize: Int, epochs: Int) = DoubleTensorAlgebra {
        fun iterBatch(x: DoubleTensor, y: DoubleTensor): Sequence<Pair<DoubleTensor, DoubleTensor>> = sequence {
            val n = x.shape[0]
            val shuffledIndices = (0 until n).shuffled()
            for (i in 0 until n step batchSize) {
                val excerptIndices = shuffledIndices.drop(i).take(batchSize).toIntArray()
                val batch = x.rowsByIndices(excerptIndices) to y.rowsByIndices(excerptIndices)
                yield(batch)
            }
        }

        for (epoch in 0 until epochs) {
            println("Epoch ${epoch + 1}/$epochs")
            for ((xBatch, yBatch) in iterBatch(xTrain, yTrain)) {
                train(xBatch, yBatch)
            }
            println("Accuracy:${accuracy(yTrain, predict(xTrain).argMax(1, true))}")
        }
    }

    fun predict(x: DoubleTensor): DoubleTensor {
        return forward(x).last()
    }

}

In [9]:
val features = 5
val sampleSize = 250
val trainSize = 180

Take features from normal distribution.

In [10]:
val x = DoubleTensorAlgebra { randomNormal(intArrayOf(sampleSize, features)) * 2.5 }

BroadcastDoubleTensorAlgebra {
    x += fromArray(
        intArrayOf(5),
        doubleArrayOf(0.0, -1.0, -2.5, -3.0, 5.5) // rows means
    )
}        

Define class like `'1'` if the sum of features > 0 and `'0'` otherwise

In [11]:
val y = DoubleTensorAlgebra { 
    fromArray(
        intArrayOf(sampleSize, 1),
        DoubleArray(sampleSize) { i ->
            if (x[i].sum() > 0.0) {
                1.0
            } else {
                0.0
            }
        }
    )
}

Split train and test (validation)

In [12]:
val trainIndices = (0 until trainSize).toList().toIntArray()
val testIndices = (trainSize until sampleSize).toList().toIntArray()

val xTrain = DoubleTensorAlgebra { x.rowsByIndices(trainIndices) }
val yTrain = DoubleTensorAlgebra { y.rowsByIndices(trainIndices) }

val xTest = DoubleTensorAlgebra { x.rowsByIndices(testIndices) }
val yTest = DoubleTensorAlgebra { y.rowsByIndices(testIndices) }

Build model.

In [13]:
val layers = mutableListOf(
    Dense(features, 64),
    ReLU(),
    Dense(64, 16),
    ReLU(),
    Dense(16, 2),
    Sigmoid()
)

val model = NeuralNetwork(layers)

Fit with it with train data.

In [14]:
model.fit(xTrain, yTrain, batchSize = 20, epochs = 10)

Epoch 1/10
Accuracy:0.6888888888888889
Epoch 2/10
Accuracy:0.9333333333333333
Epoch 3/10
Accuracy:0.95
Epoch 4/10
Accuracy:0.9611111111111111
Epoch 5/10
Accuracy:0.9777777777777777
Epoch 6/10
Accuracy:0.9833333333333333
Epoch 7/10
Accuracy:0.9722222222222222
Epoch 8/10
Accuracy:0.9888888888888889
Epoch 9/10
Accuracy:0.95
Epoch 10/10
Accuracy:0.9833333333333333


Check out accuracy on validation.

In [15]:
val prediction = model.predict(xTest) // logits

val predictionLabels = DoubleTensorAlgebra { prediction.argMax(1, true) }

In [16]:
model.accuracy(yTest, predictionLabels)

0.9428571428571428